In [1]:
import pandas as pd

citibike_data = None

for month in range(1,13):
    if month < 10:
        month_str = f'0{month}'
    else:
        month_str = f'{month}'
    print('Loading month', month_str)
    current_month = pd.read_csv(f'../dataset/2018{month_str}-citibike-tripdata.csv')
    
    if citibike_data is None:
        citibike_data = current_month
    else:
        citibike_data = citibike_data.append(current_month)
citibike_data = citibike_data.dropna()

Loading month 01
Loading month 02
Loading month 03
Loading month 04
Loading month 05
Loading month 06
Loading month 07
Loading month 08
Loading month 09
Loading month 10
Loading month 11
Loading month 12


In [2]:
citibike_data = citibike_data.dropna()

import numpy as np

citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'])
citibike_data['stoptime'] = pd.to_datetime(citibike_data['stoptime'])

citibike_data['tripduration'] = (citibike_data['stoptime'] - citibike_data['starttime']).dt.total_seconds().apply(np.floor).astype(int)

filter_columns = [
    'tripduration', 
    'starttime', 
    'stoptime', 
    'start station id',
    'start station latitude',
    'start station longitude',
    'end station id',
    'end station latitude',
    'end station longitude',
    'usertype',
    'birth year',
    'gender'
]
citibike_data = citibike_data[filter_columns]
citibike_data = citibike_data[citibike_data['tripduration'] <= 20*24*60*60]

citibike_data = citibike_data[citibike_data['end station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['start station latitude'] <= 41.13]
citibike_data = citibike_data[citibike_data['end station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['start station latitude'] >= 40.12]
citibike_data = citibike_data[citibike_data['end station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['start station longitude'] >= -74.40]
citibike_data = citibike_data[citibike_data['end station longitude'] <= -73.19]
citibike_data = citibike_data[citibike_data['start station longitude'] <= -73.19]

citibike_data['start_weekday'] = citibike_data['starttime'].dt.day_name()

citibike_data

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,40.767272,-73.993929,505.0,40.749013,-73.988484,Subscriber,1992,1,Monday
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,40.767272,-73.993929,3255.0,40.750585,-73.994685,Subscriber,1969,1,Monday
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,40.767272,-73.993929,525.0,40.755942,-74.002116,Subscriber,1956,1,Monday
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,40.767272,-73.993929,447.0,40.763707,-73.985162,Subscriber,1974,1,Monday
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,40.767272,-73.993929,3356.0,40.774667,-73.984706,Subscriber,1992,1,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016500,734,2018-12-31 23:55:06.991,2019-01-01 00:07:21.230,336.0,40.730477,-73.999061,379.0,40.749156,-73.991600,Subscriber,1986,1,Monday
1016501,2551,2018-12-31 23:55:44.307,2019-01-01 00:38:15.782,3320.0,40.794067,-73.962868,2006.0,40.765909,-73.976342,Customer,1969,0,Monday
1016502,2376,2018-12-31 23:56:52.850,2019-01-01 00:36:29.286,427.0,40.701907,-74.013942,259.0,40.701221,-74.012342,Subscriber,1979,1,Monday
1016503,2758,2018-12-31 23:58:29.212,2019-01-01 00:44:27.517,3320.0,40.794067,-73.962868,281.0,40.764397,-73.973715,Customer,1969,0,Monday


In [4]:
citibike_data = citibike_data.groupby('usertype').apply(lambda x: x.sample(500000))

In [32]:
citibike_data = citibike_data.reset_index(level=[0,1], drop=True)

In [50]:
features_df = pd.DataFrame(citibike_data)

In [34]:
from sklearn import preprocessing

unique_labels = features_df['usertype'].unique()
unique_weekdays = features_df['start_weekday'].unique()
le_ut = preprocessing.LabelEncoder()
le_ut.fit(unique_labels)
le_wd = preprocessing.LabelEncoder()
le_wd.fit(unique_weekdays)

LabelEncoder()

In [15]:
features_df['usertype'] = le_ut.transform(features_df['usertype']) 
features_df['start_weekday'] = le_wd.transform(features_df['start_weekday']) 

In [16]:
feature_columns = [
    'tripduration', 
    'start station latitude',
    'start station longitude',
    'end station latitude',
    'end station longitude',
    'birth year',
    'start_weekday',
    'gender'
]
label_column = 'usertype'

features_df = features_df[feature_columns + [label_column]]
features_df

tripduration  start station latitude  \
usertype                                                   
Customer   337148           1813               40.741740   
           214227            907               40.734546   
           996949            890               40.751873   
           1091767           955               40.714215   
           1589412          1439               40.795346   
...                          ...                     ...   
Subscriber 920290            583               40.714215   
           555693            415               40.757246   
           113461            257               40.793770   
           429001            120               40.687534   
           963030            119               40.717571   

                    start station longitude  end station latitude  \
usertype                                                            
Customer   337148                -73.994156             40.727064   
           214227                -73.990741             40.756687   
           996949                -73.977706             40.775730   
           1091767               -73.981346             40.723180   
           1589412               -73.961860             40.765909   
...                                     ...                   ...   
Subscriber 920290                -73.981346             40.708235   
           555693                -73.978059             40.748549   
           113461                -73.971888             40.790179   
           429001                -73.972652             40.691782   
           963030                -74.005549             40.718710   

                    end station longitude  birth year  start_weekday  gender  \
usertype                                                                       
Customer   337148              -73.996621        1969              2       0   
           214227              -73.982577        1983              3       1   
           996949              -73.956753        1997              0       1   
           1091767             -73.994800        1969              2       0   
           1589412             -73.976342        1993              3       1   
...                                   ...         ...            ...     ...   
Subscriber 920290              -74.005301        1979              5       1   
           555693              -73.988084        1977              4       1   
           113461              -73.972889        1995              3       2   
           429001              -73.973730        1989              5       1   
           963030              -74.009001        1980              4       2   

                    usertype  
usertype                      
Customer   337148          0  
           214227          0  
           996949          0  
           1091767         0  
           1589412         0  
...                      ...  
Subscriber 920290          1  
           555693          1  
           113461          1  
           429001          1  
           963030          1  

[1000000 rows x 9 columns]

In [17]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    features_df, 
    test_size=0.33, 
    random_state=42, 
    stratify=features_df[label_column]
)

In [18]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(
    train_set[feature_columns], 
    train_set[label_column]
)

In [20]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8061934129379347
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.80      0.81      0.81    165000
           1       0.81      0.80      0.81    165000

    accuracy                           0.81    330000
   macro avg       0.81      0.81      0.81    330000
weighted avg       0.81      0.81      0.81    330000



In [21]:
clf.feature_importances_

array([0.18369163, 0.0738087 , 0.07253992, 0.07529832, 0.07391185,
       0.0767602 , 0.03215527, 0.41183411])

In [22]:
unique_labels

array(['Customer', 'Subscriber'], dtype=object)

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_set[feature_columns], train_set[label_column])

print(clf.feature_importances_)

[0.26682315 0.02350017 0.02611431 0.01590546 0.00933267 0.20433155
 0.03662877 0.41736392]


In [25]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8245593575233905
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.92      0.66      0.77    165000
           1       0.73      0.94      0.82    165000

    accuracy                           0.80    330000
   macro avg       0.83      0.80      0.80    330000
weighted avg       0.83      0.80      0.80    330000



In [26]:
clf2 = RandomForestClassifier(max_depth=10, random_state=0)
clf2.fit(train_set[feature_columns], train_set[label_column])

print(clf2.feature_importances_)

[0.22501898 0.01274042 0.01145753 0.01361663 0.01050553 0.21173238
 0.03782429 0.47710424]


In [27]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))

0.8245593575233905
[0 1]
['Customer' 'Subscriber']
              precision    recall  f1-score   support

           0       0.92      0.66      0.77    165000
           1       0.73      0.94      0.82    165000

    accuracy                           0.80    330000
   macro avg       0.83      0.80      0.80    330000
weighted avg       0.83      0.80      0.80    330000



In [61]:
features_df = pd.DataFrame(citibike_data)

In [62]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

from sklearn import preprocessing

userttype_unique = features_df[label_column].unique()
le_pp = preprocessing.LabelEncoder()
le_pp.fit(userttype_unique)
usertype_encoded = le_pp.transform(features_df[label_column])
one_hot_encoded_usertype = to_categorical(usertype_encoded)
one_hot_encoded_usertype

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [63]:
usertype_one_hot = label_column + '_one_hot'
features_df[usertype_one_hot] = list(one_hot_encoded_usertype)
features_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,start_weekday_str,usertype_one_hot
0,1813,2018-03-31 15:31:07.212,2018-03-31 16:01:20.729,435.0,40.741740,-73.994156,3260.0,40.727064,-73.996621,0,1969,0,Saturday,Saturday,"[1.0, 0.0]"
1,907,2018-07-29 12:26:14.392,2018-07-29 12:41:21.486,285.0,40.734546,-73.990741,3466.0,40.756687,-73.982577,0,1983,1,Sunday,Sunday,"[1.0, 0.0]"
2,890,2018-07-20 15:16:38.396,2018-07-20 15:31:29.249,519.0,40.751873,-73.977706,3146.0,40.775730,-73.956753,0,1997,1,Friday,Friday,"[1.0, 0.0]"
3,955,2018-08-18 14:57:27.964,2018-08-18 15:13:23.583,502.0,40.714215,-73.981346,251.0,40.723180,-73.994800,0,1969,0,Saturday,Saturday,"[1.0, 0.0]"
4,1439,2018-06-10 18:46:47.395,2018-06-10 19:10:47.329,3341.0,40.795346,-73.961860,2006.0,40.765909,-73.976342,0,1993,1,Sunday,Sunday,"[1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,583,2018-07-31 06:57:31.954,2018-07-31 07:07:15.718,502.0,40.714215,-73.981346,306.0,40.708235,-74.005301,1,1979,1,Tuesday,Tuesday,"[0.0, 1.0]"
999996,415,2018-01-25 18:34:13.794,2018-01-25 18:41:08.824,3233.0,40.757246,-73.978059,498.0,40.748549,-73.988084,1,1977,1,Thursday,Thursday,"[0.0, 1.0]"
999997,257,2018-09-02 22:48:21.205,2018-09-02 22:52:39.055,3314.0,40.793770,-73.971888,3289.0,40.790179,-73.972889,1,1995,2,Sunday,Sunday,"[0.0, 1.0]"
999998,120,2018-08-07 20:00:31.331,2018-08-07 20:02:31.576,416.0,40.687534,-73.972652,262.0,40.691782,-73.973730,1,1989,1,Tuesday,Tuesday,"[0.0, 1.0]"


In [64]:
features_df['start_weekday'] = features_df['starttime'].dt.day_name()
features_df['start_weekday_str'] = features_df['start_weekday']
weekday_unique = features_df['start_weekday'].unique()

convert_weekdays = {
    'start_weekday': {
        'Monday': 0, 
        'Tuesday': 1, 
        'Wednesday': 2, 
        'Thursday': 3, 
        'Friday': 4, 
        'Saturday': 5, 
        'Sunday': 6}
}

features_df = features_df.replace(convert_weekdays)
features_df = features_df.drop(['start_weekday_str'], axis=1)

In [65]:
usertype_one_hot = label_column + '_one_hot'
features_df[usertype_one_hot] = list(one_hot_encoded_usertype)
features_df

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
0,1813,2018-03-31 15:31:07.212,2018-03-31 16:01:20.729,435.0,40.741740,-73.994156,3260.0,40.727064,-73.996621,0,1969,0,5,"[1.0, 0.0]"
1,907,2018-07-29 12:26:14.392,2018-07-29 12:41:21.486,285.0,40.734546,-73.990741,3466.0,40.756687,-73.982577,0,1983,1,6,"[1.0, 0.0]"
2,890,2018-07-20 15:16:38.396,2018-07-20 15:31:29.249,519.0,40.751873,-73.977706,3146.0,40.775730,-73.956753,0,1997,1,4,"[1.0, 0.0]"
3,955,2018-08-18 14:57:27.964,2018-08-18 15:13:23.583,502.0,40.714215,-73.981346,251.0,40.723180,-73.994800,0,1969,0,5,"[1.0, 0.0]"
4,1439,2018-06-10 18:46:47.395,2018-06-10 19:10:47.329,3341.0,40.795346,-73.961860,2006.0,40.765909,-73.976342,0,1993,1,6,"[1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,583,2018-07-31 06:57:31.954,2018-07-31 07:07:15.718,502.0,40.714215,-73.981346,306.0,40.708235,-74.005301,1,1979,1,1,"[0.0, 1.0]"
999996,415,2018-01-25 18:34:13.794,2018-01-25 18:41:08.824,3233.0,40.757246,-73.978059,498.0,40.748549,-73.988084,1,1977,1,3,"[0.0, 1.0]"
999997,257,2018-09-02 22:48:21.205,2018-09-02 22:52:39.055,3314.0,40.793770,-73.971888,3289.0,40.790179,-73.972889,1,1995,2,6,"[0.0, 1.0]"
999998,120,2018-08-07 20:00:31.331,2018-08-07 20:02:31.576,416.0,40.687534,-73.972652,262.0,40.691782,-73.973730,1,1989,1,1,"[0.0, 1.0]"


In [66]:
small_subset = features_df
train_set, test_set = train_test_split(
    small_subset,
    test_size=0.33, 
    random_state=42, 
    stratify=small_subset[label_column]
)
test_set

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
533157,1031,2018-07-14 17:28:00.341,2018-07-14 17:45:11.489,3618.0,40.748500,-73.941275,3514.0,40.776700,-73.927631,1,1981,1,5,"[0.0, 1.0]"
328648,3785,2018-12-18 10:23:24.657,2018-12-18 11:26:30.047,486.0,40.746201,-73.988557,3282.0,40.783070,-73.959390,0,1999,1,1,"[1.0, 0.0]"
86392,49567,2018-06-30 22:30:14.170,2018-07-01 12:16:21.373,3536.0,40.808200,-73.964100,3623.0,40.810949,-73.963400,0,1991,2,5,"[1.0, 0.0]"
614082,1265,2018-06-12 20:23:48.425,2018-06-12 20:44:53.505,3147.0,40.778012,-73.954071,3158.0,40.771639,-73.982614,1,1986,1,1,"[0.0, 1.0]"
548113,250,2018-10-21 12:10:36.587,2018-10-21 12:14:46.758,435.0,40.741740,-73.994156,470.0,40.743453,-74.000040,1,1982,1,6,"[0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362271,1541,2018-07-26 13:17:27.561,2018-07-26 13:43:08.690,3413.0,40.682755,-73.982586,3180.0,40.698780,-73.997120,0,1969,0,3,"[1.0, 0.0]"
1093,112,2018-10-13 11:24:32.991,2018-10-13 11:26:25.492,3656.0,40.723077,-73.985836,3656.0,40.723077,-73.985836,0,1975,1,5,"[1.0, 0.0]"
941967,924,2018-04-28 15:49:03.256,2018-04-28 16:04:27.474,3142.0,40.761227,-73.960940,479.0,40.760193,-73.991255,1,1987,1,5,"[0.0, 1.0]"
719799,558,2018-08-06 21:11:43.823,2018-08-06 21:21:02.057,3440.0,40.692418,-73.989495,399.0,40.688515,-73.964763,1,1988,1,0,"[0.0, 1.0]"


In [67]:
from keras import Model
from keras.layers import concatenate, Dense, Input


input_layer = Input(shape=(7,))

hidden_1 = Dense(50)(input_layer)
hidden_2 = Dense(20)(hidden_1)

classification = Dense(2, activation='softmax')(hidden_2)

model = Model(inputs=[input_layer], outputs=[classification])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                400       
_________________________________________________________________
dense_11 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 42        
Total params: 1,462
Trainable params: 1,462
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
from keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='val_accuracy', patience=5)]

train_set_labels = np.array(train_set['usertype_one_hot'].tolist())

model.fit(
    train_set[feature_columns].values, 
    train_set_labels, 
    batch_size=128, 
    epochs=10, 
    validation_split=0.3, 
    callbacks=callbacks
)

Train on 468999 samples, validate on 201001 samples
Epoch 1/10
468999/468999 [==============================] - 6s 12us/step - loss: 11.8197 - accuracy: 0.6351 - val_loss: 1.1672 - val_accuracy: 0.6927
Epoch 2/10
468999/468999 [==============================] - 5s 11us/step - loss: 5.2241 - accuracy: 0.6364 - val_loss: 0.6782 - val_accuracy: 0.7244
Epoch 3/10
468999/468999 [==============================] - 6s 12us/step - loss: 2.7510 - accuracy: 0.6446 - val_loss: 1.4742 - val_accuracy: 0.5798
Epoch 4/10
468999/468999 [==============================] - 5s 11us/step - loss: 1.6354 - accuracy: 0.6484 - val_loss: 0.6954 - val_accuracy: 0.6960
Epoch 5/10
468999/468999 [==============================] - 5s 11us/step - loss: 1.3722 - accuracy: 0.6576 - val_loss: 1.1252 - val_accuracy: 0.5591
Epoch 6/10
468999/468999 [==============================] - 5s 11us/step - loss: 1.0287 - accuracy: 0.6780 - val_loss: 0.5883 - val_accuracy: 0.7286
Epoch 7/10
468999/468999 [===========================

In [70]:
test_set_labels = np.array(test_set[usertype_one_hot].tolist())
model.evaluate([test_set[feature_columns]], test_set_labels, batch_size=128)

330000/330000 [==============================] - 1s 4us/step


[0.6135519576159391, 0.7283999919891357]

In [71]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_set[feature_columns])

print(len(test_set_labels))
print(y_pred)

y_true = [np.argmax(y, axis=None, out=None) for y in test_set_labels]
y_pred = [np.argmax(y, axis=None, out=None) for y in y_pred]

print(classification_report(y_true, y_pred))

330000
[[6.1246890e-01 3.8753107e-01]
 [9.7526509e-01 2.4734907e-02]
 [1.0000000e+00 4.8451747e-31]
 ...
 [5.7237244e-01 4.2762756e-01]
 [2.5265834e-01 7.4734163e-01]
 [8.0133575e-01 1.9866425e-01]]
              precision    recall  f1-score   support

           0       0.74      0.71      0.72    165000
           1       0.72      0.74      0.73    165000

    accuracy                           0.73    330000
   macro avg       0.73      0.73      0.73    330000
weighted avg       0.73      0.73      0.73    330000



In [72]:
print(len(train_set_labels))
print(len(train_set[feature_columns].values))

670000
670000


In [76]:
train_set

,tripduration,starttime,stoptime,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,usertype,birth year,gender,start_weekday,usertype_one_hot
413444,2603,2018-06-04 15:24:38.176,2018-06-04 16:08:01.764,3489.0,40.710700,-74.003541,427.0,40.701907,-74.013942,0,1969,0,0,"[1.0, 0.0]"
210282,1615,2018-05-25 13:10:01.453,2018-05-25 13:36:57.108,3143.0,40.776321,-73.964274,3159.0,40.774925,-73.982666,0,1980,2,4,"[1.0, 0.0]"
312939,1148,2018-08-12 13:38:55.560,2018-08-12 13:58:03.835,304.0,40.704633,-74.013617,3489.0,40.710700,-74.003541,0,1969,0,6,"[1.0, 0.0]"
639285,409,2018-04-30 08:05:25.203,2018-04-30 08:12:14.612,524.0,40.755273,-73.983169,527.0,40.744023,-73.976056,1,1988,1,0,"[0.0, 1.0]"
644034,1153,2018-10-16 08:56:58.997,2018-10-16 09:16:12.863,357.0,40.732618,-73.991580,524.0,40.755273,-73.983169,1,1980,1,1,"[0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416612,1208,2018-10-07 11:27:47.825,2018-10-07 11:47:56.671,309.0,40.714979,-74.013012,3664.0,40.720195,-74.010301,0,1969,0,6,"[1.0, 0.0]"
377862,844,2018-11-24 13:24:44.012,2018-11-24 13:38:48.457,347.0,40.728846,-74.008591,251.0,40.723180,-73.994800,0,1984,2,5,"[1.0, 0.0]"
728821,506,2018-09-05 06:54:49.499,2018-09-05 07:03:16.455,3674.0,40.701403,-73.986727,343.0,40.697940,-73.969868,1,1970,1,2,"[0.0, 1.0]"
116064,2985,2018-05-05 15:24:27.564,2018-05-05 16:14:13.119,3163.0,40.773407,-73.977825,3163.0,40.773407,-73.977825,0,1969,0,5,"[1.0, 0.0]"


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(
    train_set[feature_columns], 
    train_set[label_column]
)

In [ ]:
predicted_labels = clf.predict(test_set[feature_columns])
true_labels = test_set[label_column].tolist()

print(f1_score(true_labels, predicted_labels))

print(le_ut.transform(unique_labels))
print(unique_labels)

print(classification_report(true_labels, predicted_labels))